# Final project, Numerical Analysis 2018-2019


## Project description

In this project, we would like to compare the performance of some embarassingly simple algorithms to solve a classification problem based on the MNIST database. 

The abstract aim of the program is to write a function:

```
result = classify(image)
```

that takes as input a small grey scale image of a hand-written digit (from the MNIST database), and returns the digit corresponding to the content of the image.

An example of the images we'll be working on is the following:

![mnist examples](https://m-alcu.github.io/assets/mnist.png)

Some background on the MNIST database (from wikipedia):


## MNIST database

*From Wikipedia, the free encyclopedia*

The MNIST database (Modified National Institute of Standards and Technology database) is a large database of handwritten digits that is commonly used for training various image processing systems. The database is also widely used for training and testing in the field of machine learning. It was created by "re-mixing" the samples from NIST's original datasets. The creators felt that since NIST's training dataset was taken from American Census Bureau employees, while the testing dataset was taken from American high school students, it was not well-suited for machine learning experiments. Furthermore, the black and white images from NIST were normalized to fit into a 28x28 pixel bounding box and anti-aliased, which introduced grayscale levels.

## MNIST sample images.

The MNIST database contains 60,000 training images and 10,000 testing images. Half of the training set and half of the test set were taken from NIST's training dataset, while the other half of the training set and the other half of the test set were taken from NIST's testing dataset. There have been a number of scientific papers on attempts to achieve the lowest error rate; one paper, using a hierarchical system of convolutional neural networks, manages to get an error rate on the MNIST database of 0.23%. The original creators of the database keep a list of some of the methods tested on it. In their original paper, they use a support vector machine to get an error rate of 0.8%. An extended dataset similar to MNIST called EMNIST has been published in 2017, which contains 240,000 training images, and 40,000 testing images of handwritten digits and characters.

## Algorithm

We start by defining the distance between two images. Ideally, a distance function between two images is zero when the images are the same, and greater than zero when the images are different. 

The bigger the distance, the more different the images should be. Ideally, the distance between an image of the number `9` should be closer to an image of the number `8` than to an image of the number `1` (the digits `9` and `8`, as images, differ by the fact that the first has one closed loop, while the second has two closed loops, while the digit `1` is mostly a straight line). Two different images representing the same number should be even closer (i.e., the distance function should return a "small" number).

Given a distance and a training set of images for which we know everything, the simplest algorithm we can think of to classify an image `z`, is the following: given a set of train images (`x_train`) for which we know the digit they represent (`y_train`), measure the distance between `z` and all images in `x_train`, and classify the image `z` to represent the same digit of the image that is closest to `z` in `x_train`:

Parameters of the algorithm:

- `x_train`
- `y_train`
- a distance function `dist`  

Input of the function

- `z`

Output of the function

- `digit`

where 

```
def classify(z):
    all_distances = array([dist(x, z) for x in x_train])
    digit = y_train[argmin(all_distances)]
    return digit
```

We will experiment with different distances, and we will try to improve the algorithm above in a step by step fashon.

## Data description

Each image in the MNIST dataset represents a hand written digit, in the form of a matrix of `28x28` values between zero and one, representing gray scale values (zero = white, one = black).

We use an array of `60.000x28x28` floating point values to collect all training images, and an array of `60.000` digits containing the (correct) value of the training digits (between 0 and 9 inclusive).

The testing images are instead collected into two arrays of size `10.000x28x28` and `10.0000` respectively.

In [ ]:
%pylab inline

import keras
from keras.datasets import mnist
from keras import backend as K

img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('Training set: ', x_train.shape)

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols) #
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols)
    input_shape = (img_rows, img_cols)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 #Normalization factor
x_test /= 255

arc = load('mnist.npz')

x_train = arc['arr_0']
y_train = arc['arr_1']
x_test  = arc['arr_2']
y_test  = arc['arr_3']

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

# Plotting one image

How do we plot the images? `pyplot`, which has been imported by the first line of the previous cell, contains a command called `imshow`, that can be used to plot images. 

In this case we know it is a greyscale image, with zero representing white and one representing black, so we use a colormap that goes from white to black, i.e., `gray_r` where `_r` stands for reversed. 

In [ ]:
# Show image number 15, and write in the title what digit it should correspond to
N=15
imshow(x_train[N], cmap='gray_r')
_ = title('Hand written digit '+str(y_train[N]))

**IF YOU DON'T HAVE ENOUGH COMPUTATIONAL POWER, RUN THE EXERCISES ONLY UP TO WHAT IS SUSTAINABLE FOR YOUR PC**

General guidelines:

- Time all functions you construct, and try to make them run as fast as possible by precomputing anything that can be precomputed
- Extra points are gained if you reduce the complexity of the given algorithms in any possible way, for example by exploiting linearity, etc.
- If something takes too long to execute, make sure you time it on a smaller set of input data, and give estimates of how long it would take to run the full thing (without actually running it). Plot only the results you manage to run on your PC.

# Assignment 1

Implement the following distance functions

- d_infty $$ d_{\infty}(a,b) := \max_{i,j} |b_{ij}-a_{ij}|$$
- d_one $$ d_1(a,b) := \sum_{i,j} |b_{ij}-a_{ij}|$$
- d_two $$ d_2(a,b) := \sqrt{\sum_{i,j} |b_{ij}-a_{ij}|^2}$$

that take two `(28,28)` images in input, and return a non-negative number.

# Assignment 2

Write a function that, given a number `N`, and a distance function `dist`, computes the distance matrix D of shape `(N,N)` between the first `N` entries of `x_train`:

```
D[i,j] = dist(x_train[i], x_train[j])
```

performing the **minimum** number of operations (i.e., avoid computing a distance if it has already been computed before, i.e., keep in mind that dist(a,b) = dist(b,a)).

# Assignment 3

Compute and plot the three distance matrices

- Dinfty
- D1
- D2

for the first 100 images of the training set, using the function `imshow` applied to the three matrices

# Assignment 4

Using only a distance matrix, apply the algorithm described above and compute the efficency of the algorithm, i.e., write a function that:

Given a distance matrix with shape `(N,N)`, constructed on the first `N` samples of the `x_train` set, count the number of failures of the **leave one out** strategy, i.e., 

- set `error_counter` to zero

- for every line `i` of the matrix:

    - find the index `j` (different from `i`) for which `D[i,k] >= D[i,j]` for all `k` different from `i` and `j`.

    - if `y_train[j]` is different from `y_train[i]`, increment by one `error_counter`

- return the error: error_counter/N

- apply the function above to the 3 different distance matrices you computed before


# Assignment 5

Run the algorithm implemented above for N=100,200,400,800,1600 on the three different distances, and plot the three error rate as a function of N (i.e., compute the distance matrix, and compute the efficiency associated to the distance matrix).

You should get an error like:
```
[[ 0.58      0.17      0.17    ]
 [ 0.52      0.145     0.135   ]
 [ 0.4425    0.15      0.135   ]
 [ 0.4       0.145     0.12875 ]
 [ 0.369375  0.1025    0.09375 ]]
```
where each column represents a different norm.

** In the next assignments, optional points are given if you manage to make the algorithm run faster, by pre-computing everything you can precompute in advance **

# Assignment 6

In principle, it should be possible to decrease the error by using a better norm. From the table above, it is clear that the L2 distance works better than the L1 distance, which works better than the Linfty distance.

However, *none of these distances exploit the fact that the image is a two-dimensional object*, and that there is information also in the **neighboring** information of the pixels. 

One way to exploit this, is to interpret the image as a continuous function with values between zero and one, defined on a square domain `\Omega=[0,27]x[0,27]`.

$$ f: \Omega \to R $$

- Implement a function that computes an approximation of the $H^1$ norm distance on the renormalized images. Given two images $f_1$ and $f_2$
    - Compute $$a = \frac{f_1}{\int_\Omega f_1}$$, $$b=\frac{f_2}{\int_\Omega f_2}$$
    - Define the $H^1$ distance as
    $$
    d_{H^1}(f_1,f_2) := \sqrt{\int_\Omega |\nabla(a-b)|^2+ (a-b)^2}
    $$
    using the algorithm you prefer (or the library you prefer) to compute the gradients and the integrals. Notice that $\nabla f = (\partial f/\partial x, \partial f/\partial y)$ is a vector valued function, and $|\nabla g|^2 := (\partial g/\partial x)^2 + (\partial g/\partial y)^2$

- Compute the distance matrix and the efficiency for this distance for N=100,200,400,800,1600

## Assignment 7

An even better improvement on the previous distance function is given by the following algorithm

- Given two images $f1$ and $f2$:
    - Compute $$a = \frac{f_1}{\int_\Omega f_1}$$, $$b=\frac{f_2}{\int_\Omega f_2}$$
    - Solve 
    $$
    -\Delta \phi = a - b \qquad \text{ in } \Omega
    $$
    $$
    \phi = 0 \text{ on } \partial\Omega
    $$
    - Define the *Monge Ampere* distance
    $$
    d_{MA}(f_1,f_2) = \int_\Omega (a+b)|\nabla \phi|^2
    $$

- Compute the distance matrix and the efficiency for this distance for N=100,200,400,800,1600

## Assigment 8 (optional for DSSC, PhD and LM, Mandatory for MHPC)

Use the `BallTree` algorithm (https://en.wikipedia.org/wiki/Ball_tree), from the `sklearn` package, and construct a tree data structure **that uses one of the custom distances defined above**.

For each N in 3200,6400,12800,25600,51200, and for each distance defined above

- Build a tree using the first N entries of the training set `x_train`
- Construct a function that tests the efficiency on all the entries of the test set `x_test`:
    - for any image in `x_test`, call it `x_test[i]`, query the tree for the nearest neighbor (call it `k`), and assign as predicted digit the digit of the `x_train[k]` image, i.e., `y_train[k]`
    - check if `y_train[k]` is equal to the corresponding entry in `y_test[i]`. If not, increment a counter of the error
    - return the efficiency, i.e., `error_counter/len(x_test)`
- Plot, in a single graph, the error of each distance as a function of `N` (including labels, titles, etc.)

- Once you have the tree, experiment with different nearest neighbor algorithms, i.e., instead of taking only one nearest neighbor, take a larger number (a small number of your choice), and instead of returning the single closest digit, return the one with the largest number of occurrences. Plot the same graph you gave before, and see if you gain an improvement. Motivate all choices you have to make to get to the final answer.


**IF YOU DON'T HAVE ENOUGH COMPUTATIONAL POWER, RUN THE EXERCISES ONLY UP TO WHAT IS SUSTAINABLE FOR YOUR PC**

In [ ]:
from decimal import getcontext, Decimal

getcontext().prec = 3

In [ ]:
#Assignment 1

def d_inf(a,b):
    return norm((b-a).reshape(-1),inf)

def d_one(a,b):
    return norm((b-a).reshape(-1),1)

def d_two(a,b):
    return norm((b-a).reshape(-1),2)

In [ ]:
#Assignment 2

def distance(N,dist):
    D = zeros((N,N))
    
    for i in range(N):
        for j in range(i+1,N):
            D[i][j] = D[j][i] = dist(x_train[i],x_train[j])
            
    return D

In [ ]:
#Assignment 3

%time M_d_inf = distance(100, d_inf)
%time M_d_one = distance(100, d_one)
%time M_d_two = distance(100, d_two)

imshow(M_d_inf)
_ = title('d_inf matrix ')
show()
imshow(M_d_one)
_ = title('d_one matrix')
show()
imshow( M_d_two)
_ = title('d_two matrix')
show()

In [ ]:
#Assignment 4

N = 256

labels = y_train

print("For N=256", "\n")

%time M_d_inf = distance(N, d_inf)
%time M_d_one = distance(N, d_one)
%time M_d_two = distance(N, d_two)

print()

def efficiency(N, M, labels):
    errors = 0
    M = M[0:N,0:N]
    for idx in range(N):
        M[idx, idx] = M[idx, idx-1] 
        min_indx = np.argmin(M[idx])
        M[idx, idx] = 0

        if labels[idx] != labels[min_indx] : errors += 1
    return errors/N

print("Infinity norm efficiency: ", efficiency(N, M_d_inf, labels))
print("L1 norm efficiency: ",efficiency(N, M_d_one, labels))
print("L2 norm efficiency: ",efficiency(N, M_d_two, labels))

In [ ]:
#Assignment 5

p_fin = 10
p_in = 5
N_final = 2**p_fin

M_d_inf = distance(N_final, d_inf)
M_d_one = distance(N_final, d_one)
M_d_two = distance(N_final, d_two)

In [ ]:
eff = zeros( (5, p_fin-p_in) )

np.set_printoptions(precision=3)

for p in range(p_in,p_fin):
    eff[0,p-p_in] = efficiency(2**p, M_d_inf, labels)
    eff[1,p-p_in] = efficiency(2**p, M_d_one, labels)    
    eff[2,p-p_in] = efficiency(2**p, M_d_two, labels)
    
print("Infinity norm efficiency: ", eff[0])
print("L1 norm efficiency: ",eff[1])
print("L2 norm efficiency: ",eff[2])

plot([2**p for p in range(p_in, p_fin)], np.transpose(eff[0:3]))
_ = legend(["d_inf","d_one","d_two"])


In [ ]:
import matplotlib.pyplot as plt

plt.semilogx([2**p for p in range(p_in, p_fin)], np.transpose(eff[0:3]))
_ = legend(["d_inf","d_one","d_two"])

In [ ]:
#Assignment 6

x = np.arange(img_rows)
y = np.arange(img_cols)

grad_list = zeros((N_final, 2, img_rows, img_rows))
norm_list = zeros((N_final, img_rows, img_cols))

for i in range(N_final):
    norm_list[i] = x_train[i]/np.sum(x_train[i])
    grad_list[i] = np.gradient(norm_list[i])
    

def d_h1(a, b, g_a, g_b):
    d_h1 = sqrt(np.sum((g_a[0]-g_b[0])**2 + (g_a[1]-g_b[1])**2 + (a-b)**2))
    return d_h1

In [ ]:
M_d_h1 = zeros((N_final, N_final))
  
for i in range(N_final):
    for j in range(i+1,N_final):
        M_d_h1[i][j] = M_d_h1[j][i] = d_h1(norm_list[i], norm_list[j], grad_list[i], grad_list[j])

In [ ]:
for p in range(p_in, p_fin):
    eff[3,p-p_in] = efficiency(2**p, M_d_h1, labels)
    
print("H1 norm efficiency: ", eff[3])

In [ ]:
plot([2**p for p in range(p_in, p_fin)], np.transpose(eff[1:4]))
_ = legend(["d_one","d_two", "d_h1"])


In [ ]:
#Assignment 7

from scipy.sparse import diags, csc_matrix
from scipy.sparse.linalg import splu
from scipy.linalg import lu_solve

def laplace_build(N):
    A = diags([-1,-1,4,-1,-1], [-N,-1, 0, 1,N], shape=(N*N,N*N)).toarray()
    for i in range(1,N):
        A[i*N,(i*N)-1] = 0
        A[(i*N)-1,i*N] = 0
    return A

def LU(n):
    L = laplace_build(n)
    LAP = csc_matrix(L)
    return splu(LAP)

A = LU(len(x_train[0]) - 2)

def solve_phi(f):
    phi_temp = zeros(img_rows*img_cols)
    phi = zeros((img_cols, img_rows))
    f = f[1:-1,1:-1]
    f = f.reshape(-1)
    phi_temp = A.solve(f)
    phi[1:-1,1:-1] = phi_temp.reshape(img_rows-2, img_cols-2)
    return phi

def d_MA(a, b):
    phi = solve_phi(a-b)
    d_MA = np.sum((a+b)*((np.gradient(phi)[0])**2 + (np.gradient(phi)[1])**2))
    return d_MA

In [ ]:
M_d_MA = zeros((N_final, N_final))
  
for i in range(N_final):
    for j in range(i+1,N_final):
        M_d_MA[i][j] = M_d_MA[j][i] = d_MA(norm_list[i], norm_list[j])

In [ ]:
for p in range(p_in, p_fin):
    eff[4,p-p_in] = efficiency(2**p, M_d_MA, labels)
    
print("MA distance efficiency: ", eff[4])

In [ ]:
plot([2**p for p in range(p_in, p_fin)], np.transpose(eff[1:5]))
_ = legend(["d_one","d_two", "d_h1", "d_MA"])

In [ ]:
#KNN test

def efficiency_k(N, M, labels, k):
    
    errors = 0
    f = zeros(10)
    M = M[0:N,0:N]
    
    for idx in range(N):
        min_k_idx = np.argsort(M[idx])[1:k]
        f[:] = 0
        for i in range(k-1):
            f[labels[min_k_idx[i]]] += M[idx,min_k_idx[0]]/M[idx,min_k_idx[i]]
        if argmax(f) != labels[idx] : errors += 1
    return errors/N

In [ ]:
p_in = 3
eff_k = zeros( (5, p_fin-p_in) )
k = 6

for p in range(p_in,p_fin):
    eff_k[0,p-p_in] = efficiency_k(2**p, M_d_inf, labels, k)
    eff_k[1,p-p_in] = efficiency_k(2**p, M_d_one, labels, k)    
    eff_k[2,p-p_in] = efficiency_k(2**p, M_d_two, labels, k)
    eff_k[3,p-p_in] = efficiency_k(2**p, M_d_h1, labels, k)
    eff_k[4,p-p_in] = efficiency_k(2**p, M_d_MA, labels, k)
    

plot([2**p for p in range(p_in, p_fin)], np.transpose(eff_k[0:5]))
_ = legend(["d_inf","d_one","d_two", "d_h1", "d_MA" ])

print(eff_k)

In [ ]:
#csome test fuc
#def efficiency_k_test(N, M, labels, k):
    
    errors = 0
    f = zeros(10)
    M = M[0:N,0:N]
    
    for idx in range(1,360):
        min_k_idx = np.argsort(M[idx])[1:k]
        f[:] = 0
        for i in range(k-1):
            f[labels[min_k_idx[i]]] += 1*exp(-M[35,min_k_idx[i]]/M[35,min_k_idx[0]])
        if argmax(f) != labels[idx] : errors += 1
        print(f,argmax(f),labels[idx], errors)
    return errors/N

#efficiency_k_test(62, M_d_two, labels, 8)